**import packages**

In [ ]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import load_model
import random
from tensorflow.keras.optimizers import Adam

**load data and labels**

In [ ]:
d1 = np.load('card_aug_bw1.npy')
ln1 = np.load('labn_aug1.npy')

d2 = np.load('card_aug_bw2.npy')
ln2 = np.load('labn_aug2.npy')

d3 = np.load('card_aug_bw3.npy')
ln3 = np.load('labn_aug3.npy')

d4 = np.load('card_aug_bw4.npy')
ln4 = np.load('labn_aug4.npy')

d5 = np.load('card_aug_bw5.npy')
ln5 = np.load('labn_aug5.npy')

d6 = np.load('card_aug_bw6.npy')
ln6 = np.load('labn_aug6.npy')

d7 = np.load('card_aug_bw7.npy')
ln7 = np.load('labn_aug7.npy')

d8 = np.load('card_aug_bw8.npy')
ln8 = np.load('labn_aug8.npy')

d9 = np.load('card_aug_bw9.npy')
ln9 = np.load('labn_aug9.npy')

d10 = np.load('card_aug_bw10.npy')
ln10 = np.load('labn_aug10.npy')

tot_data = np.concatenate((d1,d2,d3,d4,d5,d6,d7,d8,d9,d10), axis=0)
tot_labn = np.concatenate((ln1,ln2,ln3,ln4,ln5,ln6,ln7,ln8,ln9,ln10), axis=0)

tot_data, tot_labn = shuffle(tot_data, tot_labn)

**preprocess data for deep learning**

In [ ]:
train_data, val_data, train_labels, val_labels = train_test_split(tot_data, tot_labn, test_size=0.1, random_state=42)

train_data = train_data.reshape(train_data.shape[0], 128, 336, 1) #reshape data for deep learning
val_data = val_data.reshape(val_data.shape[0], 128, 336, 1)

unique_labels = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18]) #vecotr of unique labels
num_classes = len(unique_labels)

label_to_index = {label: index for index, label in enumerate(unique_labels)} #dic to associate indeces to labels

# Convert the original labels to indices
label_indices_t = np.array([label_to_index[label] for label in train_labels])
label_indices_v = np.array([label_to_index[label] for label in val_labels])

one_hot_labels_t = np.eye(num_classes)[label_indices_t] #convert labels to one hot encoding
one_hot_labels_v = np.eye(num_classes)[label_indices_v]

**show random samples of the dataset**

In [ ]:
idx = random.sample(range(val_data.shape[0]), 40) #select 40 random samples

samples =val_data[idx]
lab_s = one_hot_labels_v[idx]

fig, axs = plt.subplots(5, 8, figsize=(16, 10)) # plot the samples with their labels
for i, ax in enumerate(axs.flatten()):
    ax.imshow(samples[i], cmap='gray')
    label = str(lab_s[i])
    ax.set_title(f"Label: {label}", fontsize=6)
    ax.axis('off')
plt.tight_layout()
plt.show()

**define architecture and train model**

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (21,21), activation='relu', input_shape=(128,336,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(16, (10,10), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(14, activation='softmax')
])

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model and save the training history
history = model.fit(train_data, one_hot_labels_t, epochs=45, batch_size=30, validation_data=(val_data, one_hot_labels_v))

model.save('my_model_num13fake.h5') #save trained model

plt.plot(history.history['loss'], label='Training Loss') #plot training loss
plt.plot(history.history['val_loss'], label='Test Loss') #plot validation loss
plt.xlabel('Epoch')
plt.ylabel('Loss(Numbers)')
plt.legend()
plt.show()


**display misclassified samples**

In [ ]:
pred_labels = model.predict(val_data)
# Convert predictions and true labels to class numbers
pred_classes = np.argmax(pred_labels, axis=1)
true_classes = np.argmax(one_hot_labels_v, axis=1)

misclassified_indices = np.where(pred_classes != true_classes)[0]
misclassified_images = val_data[misclassified_indices]
misclassified_true_labels = true_classes[misclassified_indices]
misclassified_pred_labels = pred_classes[misclassified_indices]

# Display misclassified samples with labels
for i in range(len(misclassified_indices)):
    plt.imshow(misclassified_images[i], cmap='gray')
    plt.title(f'True label: {misclassified_true_labels[i]+1}, Predicted label: {misclassified_pred_labels[i]+1}')
    plt.show()

12/12 [==============================] - 1s 36ms/step
[[3.8325096e-25 0.0000000e+00 0.0000000e+00 ... 1.0993308e-34
  2.2032578e-28 1.4512494e-38]
 [3.2162102e-14 1.8562786e-12 3.6391835e-15 ... 8.1110585e-09
  2.4863273e-11 6.6143732e-13]
 [1.7055545e-14 6.2986720e-13 1.0000000e+00 ... 5.1196255e-19
  1.2593858e-17 5.0941159e-15]
 ...
 [1.3982179e-13 6.7022138e-10 1.0000000e+00 ... 4.9305812e-19
  3.4185779e-16 4.8744337e-14]
 [2.2096294e-13 1.6446806e-17 1.9301047e-14 ... 6.2584607e-16
  1.4663264e-13 1.2110292e-16]
 [0.0000000e+00 0.0000000e+00 3.3624647e-28 ... 6.5048769e-35
  1.9333337e-20 0.0000000e+00]]
